# Imports

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

awards_players_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/awards_players_cleaned.csv')
coaches_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/coaches_cleaned.csv')
players_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/players_cleaned.csv')
players_teams_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/players_teams_cleaned.csv')
series_post_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/series_post_cleaned.csv')
teams_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/teams_cleaned.csv')
teams_post_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/teams_post_cleaned.csv')

# Obtain data from only 9 years

In [65]:
awards_players_model = awards_players_cleaned[awards_players_cleaned['year'] != 10]
coaches_model = coaches_cleaned[coaches_cleaned['year'] != 10]
players_teams_model = players_teams_cleaned[players_teams_cleaned['year'] != 10]
series_post_model = series_post_cleaned[series_post_cleaned['year'] != 10]
teams_model = teams_cleaned[teams_cleaned['year'] != 10]
teams_post_model = teams_post_cleaned[teams_post_cleaned['year'] != 10]

os.makedirs('../data/basketballPlayoffs_model', exist_ok=True)

awards_players_model.to_csv('../data/basketballPlayoffs_model/awards_players_model.csv', index=False)
coaches_model.to_csv('../data/basketballPlayoffs_model/coaches_model.csv', index=False)
players_cleaned.to_csv('../data/basketballPlayoffs_model/players_model.csv', index=False)
players_teams_model.to_csv('../data/basketballPlayoffs_model/players_teams_model.csv', index=False)
series_post_model.to_csv('../data/basketballPlayoffs_model/series_post_model.csv', index=False)
teams_model.to_csv('../data/basketballPlayoffs_model/teams_model.csv', index=False)
teams_post_model.to_csv('../data/basketballPlayoffs_model/teams_post_model.csv', index=False)

# Overalls from only 9 years

In [66]:
players_overall_avg = players_teams_model.groupby('playerID')['OVERALL'].mean().reset_index()

players_overall_avg.rename(columns={'OVERALL': 'OVERALL_ALL_TIME'}, inplace=True)

players_overall_avg['OVERALL_ALL_TIME'] = players_overall_avg['OVERALL_ALL_TIME'].round(1)

players_overall_avg.to_csv('../data/basketballPlayoffs_model/players_overall_all_time_model.csv', index=False)

In [67]:
rookie_players = players_teams_model[players_teams_model['is_rookie'] == 1]

rookie_overall_avg = rookie_players['OVERALL'].mean().round(1)

rookie_overall_avg_df = pd.DataFrame({'rookie_overall_avg': [rookie_overall_avg]})

rookie_overall_avg_df.to_csv('../data/basketballPlayoffs_model/rookie_overall_avg_model.csv', index=False)

In [68]:
rookie_coaches = coaches_model[coaches_model['is_rookie'] == 1]

rookie_overall_coaches_avg = rookie_coaches['OVERALL'].mean().round(1)

rookie_overall_coaches_avg_df = pd.DataFrame({'rookie_overall_coaches_avg': [rookie_overall_coaches_avg]})

rookie_overall_coaches_avg_df.to_csv('../data/basketballPlayoffs_model/rookie_overall_coaches_avg_model.csv', index=False)

In [69]:
coaches_overall_avg = coaches_model.groupby('coachID')['OVERALL'].mean().reset_index()

coaches_overall_avg.rename(columns={'OVERALL': 'OVERALL_ALL_TIME'}, inplace=True)

coaches_overall_avg['OVERALL_ALL_TIME'] = coaches_overall_avg['OVERALL_ALL_TIME'].round(1)

coaches_overall_avg.to_csv('../data/basketballPlayoffs_model/coaches_overall_all_time_model.csv', index=False)

# Obtain ranks from the 10th year to compare

In [70]:
playoffs_10th_year = teams_cleaned[teams_cleaned['year'] == 10][['tmID', 'playoff']]

playoffs_10th_year.to_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv', index=False)

# Obtain teams line up and coach from the 10th year

In [71]:
coaches_model_10th_year = coaches_cleaned[coaches_cleaned['year'] == 10][['coachID', 'tmID',]]

coaches_model_10th_year.to_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv', index=False)

In [72]:
players_model_10th_year = players_teams_cleaned[players_teams_cleaned['year'] == 10][['playerID','tmID']]

players_model_10th_year = players_model_10th_year.sort_values(by='tmID')

players_model_10th_year.to_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv', index=False)

# Add overalls calculated from 9 years to the 10th year players and coach

In [73]:
players_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv')
players_overall_all_time_model = pd.read_csv('../data/basketballPlayoffs_model/players_overall_all_time_model.csv')

rookie_avg = 5.6 

players_model_10th_year = players_model_10th_year.merge(
    players_overall_all_time_model[['playerID', 'OVERALL_ALL_TIME']],
    on='playerID',
    how='left'
).fillna({'OVERALL_ALL_TIME': rookie_avg})

players_model_10th_year.rename(columns={'OVERALL_ALL_TIME': 'OVERALL'}, inplace=True)

players_model_10th_year.to_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv', index=False)

In [74]:
coaches_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv')
coaches_overall_all_time_model = pd.read_csv('../data/basketballPlayoffs_model/coaches_overall_all_time_model.csv')

rookie_avg_coach = 8.9 

coaches_model_10th_year = coaches_model_10th_year.merge(
    coaches_overall_all_time_model[['coachID', 'OVERALL_ALL_TIME']],
    on='coachID',
    how='left'
).fillna({'OVERALL_ALL_TIME': rookie_avg_coach})

coaches_model_10th_year.rename(columns={'OVERALL_ALL_TIME': 'OVERALL'}, inplace=True)

coaches_model_10th_year.to_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv', index=False)


# Given the teams players composition and their respective overalls calculate the team overall (mean of all players)

In [75]:
players_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv')

team_overall = players_model_10th_year.groupby('tmID')['OVERALL'].mean().reset_index()

team_overall['overall_team'] = team_overall['OVERALL'].round(1)

team_overall = team_overall.drop(columns=['OVERALL'])

team_overall.to_csv('../data/basketballPlayoffs_model/team_overall.csv', index=False)

# Given the team coaches and their respective overalls calculate the coach overall (mean of all coaches(1 or more then 1))

In [76]:
coaches_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv')

team_overall_coaches = coaches_model_10th_year.groupby('tmID')['OVERALL'].mean().reset_index()

team_overall_coaches['overall_team_coach'] = team_overall_coaches['OVERALL'].round(1)

team_overall_coaches = team_overall_coaches.drop(columns=['OVERALL'])

team_overall_coaches.to_csv('../data/basketballPlayoffs_model/team_overall_coaches.csv', index=False)

# Create team final overall ( 70% team + 30% coach)

In [77]:
team_overall = pd.read_csv('../data/basketballPlayoffs_model/team_overall.csv')
team_overall_coach = pd.read_csv('../data/basketballPlayoffs_model/team_overall_coaches.csv')
teams_model = pd.read_csv('../data/basketballPlayoffs_model/teams_model.csv')

merged_df = pd.merge(team_overall, team_overall_coach, on='tmID')

merged_df['overall_team_final'] = (merged_df['overall_team'] * 0.9) + (merged_df['overall_team_coach'] * 0.1)
merged_df['overall_team_final'] = merged_df['overall_team_final'].round(2)

merged_df = pd.merge(merged_df, teams_model[['tmID', 'confID']], on='tmID', how='left')

final_df = merged_df[['tmID', 'confID', 'overall_team_final']]

final_df = final_df.drop_duplicates(subset='tmID')

final_df = final_df.sort_values(by='overall_team_final', ascending=True)

final_df.to_csv('../data/basketballPlayoffs_model/team_final_overall.csv', index=False)

# Accuracy

In [78]:
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')

top_teams_by_conf = team_final_overall.sort_values(by='overall_team_final', ascending=False).groupby('confID').head(4)

print("Teams that go to the playoffs acording to the previsions:\n")
print(top_teams_by_conf)

comparison = top_teams_by_conf.merge(playoffs_10th_year, on='tmID', how='left')

correct_predictions = comparison[comparison['playoff'] == 1]

print("\nTeams that are in the playoffs and in the previsions:\n")
print(correct_predictions)

accuracy = len(correct_predictions) / len(top_teams_by_conf) * 100
print(f"\nThe accuracy is {accuracy:.2f}%")


Teams that go to the playoffs acording to the previsions:

    tmID  confID  overall_team_final
12     8       1               11.17
11    16       1                9.13
10     5       0                8.95
9     13       1                8.68
8     17       1                8.54
5     11       0                8.28
4      7       0                8.21
3     19       0                8.18

Teams that are in the playoffs and in the previsions:

   tmID  confID  overall_team_final  playoff
0     8       1               11.17        1
1    16       1                9.13        1
2     5       0                8.95        1
3    13       1                8.68        1
4    17       1                8.54        1
6     7       0                8.21        1
7    19       0                8.18        1

The accuracy is 87.50%


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

FEATURES = ['o_pts', 'd_pts', 'won', 'lost', 'homeW', 'homeL', 
            'awayW', 'awayL', 'confW', 'confL', 'tmORB', 'tmDRB', 'tmTRB',
            'overall_team', 'overall_team_coach', 'overall_team_final']

def prepare_data(teams_model, team_final_overall, team_overall, team_overall_coaches):
    merged_data = teams_model.merge(team_final_overall[['tmID', 'overall_team_final']], 
                                  on='tmID', 
                                  how='left')
    
    merged_data = merged_data.merge(team_overall[['tmID', 'overall_team']], 
                                  on='tmID', 
                                  how='left')
    
    merged_data = merged_data.merge(team_overall_coaches[['tmID', 'overall_team_coach']], 
                                  on='tmID', 
                                  how='left')
    
    X = merged_data[FEATURES]
    y = merged_data['playoff']
    
    imputer = SimpleImputer(strategy='mean')
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    return X, y, imputer

def train_knn_model(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Modified parameters to be more distinct from SVM
    knn = KNeighborsClassifier(
        n_neighbors=7,           # Increased from 5
        weights='distance',      # Keep distance weighting
        metric='manhattan',      # Changed to manhattan distance
        p=1,                    # Changed to match manhattan distance
        n_jobs=-1
    )
    
    # Add cross-validation score
    from sklearn.model_selection import cross_val_score
    cv_scores = cross_val_score(knn, X_train_scaled, y_train, cv=5)
    print(f"Cross-validation scores: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")
    
    knn.fit(X_train_scaled, y_train)
    return knn, scaler

def predict_playoffs(teams_10th_year, team_final_overall, team_overall, 
                    team_overall_coaches, knn_model, scaler, imputer):
    merged_10th = teams_10th_year.merge(team_final_overall[['tmID', 'overall_team_final']], 
                                       on='tmID', 
                                       how='left')
    
    merged_10th = merged_10th.merge(team_overall[['tmID', 'overall_team']], 
                                   on='tmID', 
                                   how='left')
    
    merged_10th = merged_10th.merge(team_overall_coaches[['tmID', 'overall_team_coach']], 
                                   on='tmID', 
                                   how='left')
    
    X_10th = merged_10th[FEATURES]
    X_10th = pd.DataFrame(imputer.transform(X_10th), columns=X_10th.columns)
    X_10th_scaled = scaler.transform(X_10th)
    
    return knn_model.predict(X_10th_scaled)

# Load and prepare data
teams_model = pd.read_csv('../data/basketballPlayoffs_model/teams_model.csv')
teams_10th_year = teams_cleaned[teams_cleaned['year'] == 10]
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
team_overall = pd.read_csv('../data/basketballPlayoffs_model/team_overall.csv')
team_overall_coaches = pd.read_csv('../data/basketballPlayoffs_model/team_overall_coaches.csv')

# Prepare the data
X, y, imputer = prepare_data(teams_model, team_final_overall, team_overall, team_overall_coaches)

# Split data using same ratio as SVM
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Train model
knn_model, scaler = train_knn_model(X_train, y_train)

# Test model
X_test_scaled = scaler.transform(X_test)
y_pred_test = knn_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test set accuracy: {test_accuracy*100:.2f}%")

# Make predictions for 10th year
predictions_10th = predict_playoffs(teams_10th_year, team_final_overall, team_overall, 
                                  team_overall_coaches, knn_model, scaler, imputer)

# Evaluate results
playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')
actual_playoffs = playoffs_10th_year['playoff'].values

results_df = pd.DataFrame({
    'Team_ID': teams_10th_year['tmID'].values,
    'Conference': teams_10th_year['confID'].values,
    'Predicted': predictions_10th,
    'Actual': actual_playoffs
})

print("\nOverall Predictions vs Actual Results:")
print(results_df)

print("\nDetailed Classification Report:")
print(classification_report(actual_playoffs, predictions_10th))

accuracy = accuracy_score(actual_playoffs, predictions_10th) * 100
print(f"\nOverall Accuracy: {accuracy:.2f}%")

Cross-validation scores: 0.926 (+/- 0.112)
Test set accuracy: 90.00%

Overall Predictions vs Actual Results:
    Team_ID  Conference  Predicted  Actual
0         0           0          1       1
1         2           0          1       0
2         4           0          1       0
3         5           0          1       1
4         7           0          1       1
5         8           1          1       1
6        10           1          0       0
7        11           0          0       0
8        13           1          1       1
9        15           1          0       0
10       16           1          0       1
11       17           1          1       1
12       19           0          1       1

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.78      0.88      0.82         8

    accuracy                           0.77        13
   macro avg       0.76      0.74     

In [80]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
5

FEATURES = ['o_pts', 'd_pts', 'won', 'lost', 'homeW', 'homeL', 
            'awayW', 'awayL', 'confW', 'confL', 'tmORB', 'tmDRB', 'tmTRB',
            'overall_team', 'overall_team_coach', 'overall_team_final']

## Data Preparation:

Merges multiple datasets containing team stats and coach ratings into one dataset, selects relevant features (X) for prediction, and the target variable (y). It also imputes missing feature values with the mean.

In [81]:
def prepare_data(teams_model, team_final_overall, team_overall, team_overall_coaches):
    merged_data = teams_model.merge(team_final_overall[['tmID', 'overall_team_final']], 
                                    on='tmID', 
                                    how='left')
    merged_data = merged_data.merge(team_overall[['tmID', 'overall_team']], 
                                    on='tmID', 
                                    how='left')
    merged_data = merged_data.merge(team_overall_coaches[['tmID', 'overall_team_coach']], 
                                    on='tmID', 
                                    how='left')
    
    X = merged_data[FEATURES]
    y = merged_data['playoff']
    
    imputer = SimpleImputer(strategy='mean')
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    return X, y, imputer

## Model Training (SVM):

Standardizes  the features to ensure they are on a consistent scale and trains an SVM classifier with an RBF kernel to predict playoff success. The trained model and scaler are returned for future predictions.

We choce the RBF kernel because it works well for non-linear data like basketball statistics

We selected C=0.5 because it ensures that the model does not become too simple or too complex.

We selected gamma=0.1 because it provides a moderate influence from each training point.

In [82]:
def train_svm_model(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    svm = SVC(kernel='rbf', C=0.5, gamma=0.1, probability=True)
    svm.fit(X_train_scaled, y_train)
    
    return svm, scaler

## Prediction Function:

Prepares the data for prediction for a new set of teams, merges the relevant datasets, imputes missing values, scales the features using the trained scaler, and applies the trained SVM model to make predictions.

In [83]:
def predict_playoffs(teams_10th_year, team_final_overall, team_overall, 
                    team_overall_coaches, svm_model, scaler, imputer):
    merged_10th = teams_10th_year.merge(team_final_overall[['tmID', 'overall_team_final']], 
                                        on='tmID', 
                                        how='left')
    merged_10th = merged_10th.merge(team_overall[['tmID', 'overall_team']], 
                                    on='tmID', 
                                    how='left')
    merged_10th = merged_10th.merge(team_overall_coaches[['tmID', 'overall_team_coach']], 
                                    on='tmID', 
                                    how='left')
    
    X_10th = merged_10th[FEATURES]
    X_10th = pd.DataFrame(imputer.transform(X_10th), columns=X_10th.columns)
    X_10th_scaled = scaler.transform(X_10th)
    
    return svm_model.predict(X_10th_scaled)

## Main Execution Flow:

Loads the data, prepares it for training and testing, splits it into training and test sets, trains the SVM model, evaluates its accuracy, makes predictions for the 10th year, and compares predictions with actual results. Outputs a detailed comparison of predictions and actual outcomes, including accuracy metrics.

In [84]:
teams_model = pd.read_csv('../data/basketballPlayoffs_model/teams_model.csv')
teams_10th_year = teams_cleaned[teams_cleaned['year'] == 10]
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
team_overall = pd.read_csv('../data/basketballPlayoffs_model/team_overall.csv')
team_overall_coaches = pd.read_csv('../data/basketballPlayoffs_model/team_overall_coaches.csv')

X, y, imputer = prepare_data(teams_model, team_final_overall, team_overall, team_overall_coaches)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

svm_model, scaler = train_svm_model(X_train, y_train)

X_test_scaled = scaler.transform(X_test)
y_pred_test = svm_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test set accuracy: {test_accuracy*100:.2f}%")

predictions_10th = predict_playoffs(teams_10th_year, team_final_overall, team_overall, 
                                  team_overall_coaches, svm_model, scaler, imputer)

playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')
actual_playoffs = playoffs_10th_year['playoff'].values

results_df = pd.DataFrame({
    'Team_ID': teams_10th_year['tmID'].values,
    'Conference': teams_10th_year['confID'].values,
    'Predicted': predictions_10th,
    'Actual': actual_playoffs
})

print("\nOverall Predictions vs Actual Results:")
print(results_df)

print("\nDetailed Classification Report:")
print(classification_report(actual_playoffs, predictions_10th))

accuracy = accuracy_score(actual_playoffs, predictions_10th) * 100
print(f"\nOverall Accuracy: {accuracy:.2f}%")

Test set accuracy: 90.00%

Overall Predictions vs Actual Results:
    Team_ID  Conference  Predicted  Actual
0         0           0          1       1
1         2           0          0       0
2         4           0          1       0
3         5           0          1       1
4         7           0          1       1
5         8           1          1       1
6        10           1          0       0
7        11           0          0       0
8        13           1          1       1
9        15           1          0       0
10       16           1          0       1
11       17           1          1       1
12       19           0          0       1

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.86      0.75      0.80         8

    accuracy                           0.77        13
   macro avg       0.76      0.78      0.76        13
weighted avg       0.78    